# 1D Maze Place Cell Analysis

Workflow for analyzing place cells in a 3D maze with tube corridors.
Tube traversals are serialized onto a concatenated 1D axis and analyzed
using the same metrics (SI, stability, place fields) as the 2D pipeline.

In [ ]:
from pathlib import Path

from tqdm.auto import tqdm

from placecell.dataset import PlaceCellDataset
from placecell.visualization import (
    plot_occupancy_preview_1d,
    plot_rate_map_1d,
    plot_shuffle_test_1d,
    plot_summary_scatter,
    plot_diagnostics,
)

## Configuration

Point to a maze analysis config and a per-session data paths YAML.

In [ ]:
CONFIG_ID = "pcell_maze_config"
DATA_PATH = Path(
    "/Volumes/ProcData/minizero_analysis/202512round/"
    "202512_analysis_3dmaze/20251219/WL25/WL25_20251219.yaml"
)

ds = PlaceCellDataset.from_yaml(CONFIG_ID, DATA_PATH)
print(type(ds).__name__)  # Should print 'MazeDataset'

## Step 1: Load & Preprocess

In [ ]:
ds.load()

In [ ]:
ds.preprocess_behavior()

In [ ]:
ds.deconvolve(progress_bar=tqdm)

## Step 2: Match Events & Occupancy

In [ ]:
ds.match_events()
ds.compute_occupancy()

In [ ]:
plot_occupancy_preview_1d(
    ds.trajectory_1d_filtered,
    ds.occupancy_time,
    ds.valid_mask,
    ds.edges_1d,
    tube_boundaries=ds.tube_boundaries,
    tube_labels=ds.maze_cfg.tube_order,
);

## Step 3: Analyze Units

In [ ]:
ds.analyze_units(progress_bar=tqdm)

In [ ]:
summary = ds.summary()
print(summary)

In [ ]:
plot_summary_scatter(ds.unit_results, p_value_threshold=ds.spatial_1d.p_value_threshold);

In [ ]:
plot_diagnostics(ds.unit_results, p_value_threshold=ds.spatial_1d.p_value_threshold);

## Shuffle tests & population rate map (Guo et al. 2023 Fig 3J)

Three-panel figure:
1. **Population heatmap** — rate maps of all place cells sorted by peak position
2. **Significance** — observed spatial information vs circular-shift shuffle distribution
3. **Stability** — observed split-half correlation vs circular-shift shuffle distribution

In [ ]:
plot_shuffle_test_1d(
    ds.unit_results,
    ds.edges_1d,
    p_value_threshold=ds.spatial_1d.p_value_threshold,
    tube_boundaries=ds.tube_boundaries,
    tube_labels=ds.maze_cfg.tube_order,
);

## Browse individual unit rate maps

In [ ]:
# Show rate maps for significant + stable units
p_thresh = ds.spatial_1d.p_value_threshold
for uid, res in ds.unit_results.items():
    is_sig = res.p_val < p_thresh
    is_stable = not __import__('numpy').isnan(res.stability_p_val) and res.stability_p_val < p_thresh
    if is_sig and is_stable:
        plot_rate_map_1d(
            res.rate_map,
            ds.edges_1d,
            tube_boundaries=ds.tube_boundaries,
            tube_labels=ds.maze_cfg.tube_order,
            title=f"Unit {uid} (SI={res.si:.3f}, p={res.p_val:.3f}, stab_z={res.stability_z:.2f})",
        )

## Save Bundle

In [ ]:
# ds.save_bundle(DATA_PATH.parent / f"{DATA_PATH.stem}.pcellbundle")